## Sending Data to Firebase

#### Import Libraries

In [ ]:
import firebase_admin
import paho.mqtt.client as mqtt
import paho.mqtt.client as mqtt
from firebase_admin import credentials  
from firebase_admin import db

from datetime import datetime
import plotly.graph_objects as go

#### Initialize Firebase Admin SDK

In [ ]:
cred = credentials.Certificate("D:/Projects/Project 4/IOT/Project/smart-home-69c35-firebase-adminsdk-71ryr-997205ee4f.json")
firebase_admin.initialize_app(cred, {'databaseURL': 'https://smart-home-69c35-default-rtdb.firebaseio.com/'})

In [ ]:
ref = db.reference('/')

In [ ]:
# MQTT broker address
broker_address = "192.168.137.1"

# MQTT broker port
port = 1883
global payload
payload = None  # Initialize payload
fan_topic="fan_state"
gas_topic="gas"
buzz_topic="buzz"
topic = "temp_sens"
# Quality of Service (QoS)
qos = 0
temperature = []
gas=[]
buzz=[]

def on_message(client, userdata, message):
    global temperature
    global gas
    global buzz
    topic = message.topic
    payload = message.payload.decode()
    print(topic + ": " + payload)
    if topic == "temp_sens":
            temperature = payload
            print("New fire message:", temperature)
            send_temp_to_firebase(temperature)
    if topic == "gas":
        gas = payload
        print(gas)
        send_gas_to_firebase(gas) 
    if topic == "buzz":
           buzz=payload
           print(buzz)

def send_temp_to_firebase(sensor_data):
    time = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    ref.child('room/temp_sensor').push({'timestamp': time, 'temp': sensor_data})
    print("Sensor data sent to Firebase:", sensor_data)

def send_gas_to_firebase(sensor_data):
    time = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    ref.child('room/gas_sensor').push({'timestamp': time, 'gas': sensor_data})
    print("Sensor data sent to Firebase:", sensor_data)
# Create an MQTT client instance
client = mqtt.Client("kolorayh")

# Set the callback function for incoming messages
client.on_message = on_message


# Connect to the MQTT broker
client.connect(broker_address, port)

# Subscribe to the specified topics
client.subscribe("temp_sens", qos)
client.subscribe("gas", qos)
client.subscribe("buzz", qos)


# Start the MQTT client loop to receive messages
client.loop_start()

#### Get the Data from `temp_sensor` Node

In [ ]:
def get_temp_values():
    temp_sensor = ref.child('room/temp_sensor').get()

    timestamp = []
    temp = []
    
    for key, val in temp_sensor.items():
        temp.append(val['temp'])
        timestamp.append(val['timestamp'])

    return timestamp, temp
def get_gas_values():
    gas_sensor = ref.child('room/gas_sensor').get()

    timestamp = []
    gas = []
    
    for key, val in gas_sensor.items():
        gas.append(val['gas'])
        timestamp.append(val['timestamp'])

    return timestamp, gas


In [ ]:
def get_temp_values():
    temp_sensor = ref.child('room/temp_sensor').get()
    
    if temp_sensor is None:
        raise ValueError("temp_sensor cannot be None")

    print("Retrieved temp_sensor data:", temp_sensor)  # Log the retrieved data

    timestamp = []
    temp = []

    for key, val in temp_sensor.items():
        if isinstance(val, dict) and 'temp' in val and 'timestamp' in val:
            temp.append(val['temp'])
            timestamp.append(val['timestamp'])
        else:
            print(f"Unexpected value for key {key}: {val}")  # Debugging output

    return timestamp, temp

# Example usage (assuming ref is properly initialized and pointing to the correct database reference)

timestamp, temp = get_temp_values()




In [ ]:
def get_gas_values():
    gas_sensor = ref.child('room/gas_sensor').get()

    if gas_sensor is None:
        raise ValueError("gas_sensor cannot be None")

    print("Retrieved gas_sensor data:", gas_sensor)  # Log the retrieved data

    timestamp = []
    gas = []

    for key, val in gas_sensor.items():
        if isinstance(val, dict) and 'gas' in val and 'timestamp' in val:
            gas.append(val['gas'])
            timestamp.append(val['timestamp'])
        else:
            print(f"Unexpected value for key {key}: {val}")  # Debugging output

    return timestamp, gas

# Example usage (assuming ref is properly initialized and pointing to the correct database reference)
timestamp, gas = get_gas_values()



#### Visualizing Data

In [ ]:
def viz_temp(timestamp, temp):
    data = go.Scatter(x=timestamp, y=temp)
    fig = go.Figure(data=data)

    fig.layout.title = 'Temprature Over Day'
    fig.layout.xaxis.title = 'Timestamp'
    fig.layout.yaxis.title = 'Temprature'

    return fig

In [ ]:
def viz_gas(timestamp, gas):
    data = go.Scatter(x=timestamp, y=gas)
    fig = go.Figure(data=data)

    fig.layout.title = 'Temprature Over Day'
    fig.layout.xaxis.title = 'Timestamp'
    fig.layout.yaxis.title = 'Gas Value'

    return fig

#### Simple Dashboard Using Plotly and Flask

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
# Import necessary modules from Flask
from flask import Flask, render_template, request, jsonify 

# Create a Flask application object
app = Flask(__name__)
nowtemp= temp[-1]
nowgas = gas[-1]
# Define a route for the root URL ('/')
@app.route('/')
def index():
    # Get timestamp and temperature values
    timestamp, temp = get_temp_values()
    timestamp, gas= get_gas_values()
    
    # Visualize temperature data and get a Plotly figure
    fig = viz_temp(timestamp, temp)
    fig2 = viz_gas(timestamp, gas)
    # Convert the Plotly figure to JSON format
    fig_json = fig.to_json()
    fig2_json=fig2.to_json()

    # Render the HTML template 'index.html', passing the Plotly figure JSON data
    return render_template('kolorayh.html', fig_json=fig_json,nowtemp=nowtemp,fig2_json=fig2_json,nowgas=nowgas)
@app.route('/toggle', methods=['POST'])
def toggle():
    data = request.json
    if 'state' in data:
        if data['state']:
            print("Button turned ON")
            client.publish(fan_topic, "on", qos)
            client.loop_start()
        else:
            print("Button turned OFF")
            client.publish(fan_topic, "off", qos)
            client.loop_start()
    return 'OK'
# Check if the script is being run directly
if __name__ == '__main__':
    # Start the Flask server
    app.run()